In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.diagnostic import het_breuschpagan


In [3]:

# CSV laden
df = pd.read_csv("NEWmergedpop.csv")

# Sicherstellen, dass SIDS und LDCs als 0/1 Dummy kodiert sind
df["SIDS"] = df["SIDS"].astype(int)
df["LDCs"] = df["LDCs"].astype(int)

# Relevante Spalten für beide Modelle
all_vars = ['SIDS', 'LDCs', 'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
            'GDPCap', 
            #'NDGAIN',
           'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']

# RPPOP-spezifisch
vars_rp = ['AdaptationMitigationRatio'] + all_vars
X_rp = df[vars_rp].dropna()
X_rp = sm.add_constant(X_rp)

# FAPOP-spezifisch (ohne AdaptationMitigationRatio)
X_fa = df[all_vars].dropna()
X_fa = sm.add_constant(X_fa)


In [29]:
#VIF-berechnen

def calculate_vif(df_features):
    # Nur numerische Spalten behalten
    df_features = df_features.select_dtypes(include='number')

    vif_data = pd.DataFrame()
    vif_data["Variable"] = df_features.columns
    vif_data["VIF"] = [variance_inflation_factor(df_features.values, i)
                       for i in range(df_features.shape[1])]

    # Nach VIF-Werten absteigend sortieren
    return vif_data.sort_values(by="VIF", ascending=False)


print("VIF für RPPOP-Modell:")
print(calculate_vif(X_rp.drop(columns='const')))

print("\nVIF für FAPOP-Modell:")
print(calculate_vif(X_fa.drop(columns='const')))


VIF für RPPOP-Modell:
                     Variable        VIF
7                      NDGAIN  18.541749
4                      GRadUn   7.257712
3                      CO2BIP   5.775760
6                    Ambition   5.219776
0   AdaptationMitigationRatio   3.542846
2                        LDCs   2.482293
5                  AvgChange%   2.021333
8                    Gov_PCA1   1.796875
1                        SIDS   1.417868
10                   δCO2/GDP   1.363945
11            Ambitionsfaktor   1.175227
9                    Gov_PCA2   1.132183

VIF für FAPOP-Modell:
           Variable        VIF
6            NDGAIN  14.678325
3            GRadUn   7.182128
5          Ambition   5.142407
2            CO2BIP   3.686115
1              LDCs   2.323353
4        AvgChange%   1.872657
7          Gov_PCA1   1.639429
0              SIDS   1.368783
9          δCO2/GDP   1.306570
8          Gov_PCA2   1.222792
10  Ambitionsfaktor   1.128056


In [23]:
# Korrelation zwischen LogGDP und NDGAIN berechnen
correlation = df[["GDPCap", "NDGAIN"]].corr()

correlation

,GDPCap,NDGAIN
GDPCap,1.000000,0.764231
NDGAIN,0.764231,1.000000


In [30]:

# CSV laden
df = pd.read_csv("NEWmergedpop.csv")

# Dummy-Variablen korrekt typisieren
df["SIDS"] = df["SIDS"].astype(int)
df["LDCs"] = df["LDCs"].astype(int)

# Funktion zur VIF-Berechnung
def calculate_vif(df_features):
    df_features = df_features.select_dtypes(include='number')
    vif_data = pd.DataFrame()
    vif_data["Variable"] = df_features.columns
    vif_data["VIF"] = [variance_inflation_factor(df_features.values, i)
                       for i in range(df_features.shape[1])]
    return vif_data.sort_values(by="VIF", ascending=False)

# Relevante Variablen definieren
all_vars = ['SIDS', 'LDCs', 'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
            'GDPCap', 'NDGAIN',
            'va', 'pv', 'ge', 'rl', 'cc', 'δCO2/GDP', 'Ambitionsfaktor']
vars_rp = ['AdaptationMitigationRatio'] + all_vars

# RPPOP ohne NDGAIN
X_rp_ohne_ndgain = df[vars_rp].drop(columns=["NDGAIN"]).dropna()
X_rp_ohne_ndgain = sm.add_constant(X_rp_ohne_ndgain)
print("\n📉 VIF für RPPOP-Modell ohne NDGAIN:")
print(calculate_vif(X_rp_ohne_ndgain.drop(columns='const')))

# RPPOP ohne LogGDPCap
X_rp_ohne_gdp = df[vars_rp].drop(columns=["GDPCap"]).dropna()
X_rp_ohne_gdp = sm.add_constant(X_rp_ohne_gdp)
print("\n📉 VIF für RPPOP-Modell ohne GDPCap:")
print(calculate_vif(X_rp_ohne_gdp.drop(columns='const')))

# FAPOP ohne NDGAIN
X_fa_ohne_ndgain = df[all_vars].drop(columns=["NDGAIN"]).dropna()
X_fa_ohne_ndgain = sm.add_constant(X_fa_ohne_ndgain)
print("\n📉 VIF für FAPOP-Modell ohne NDGAIN:")
print(calculate_vif(X_fa_ohne_ndgain.drop(columns='const')))

# FAPOP ohne LogGDPCap
X_fa_ohne_gdp = df[all_vars].drop(columns=["GDPCap"]).dropna()
X_fa_ohne_gdp = sm.add_constant(X_fa_ohne_gdp)
print("\n📉 VIF für FAPOP-Modell ohne GDPCap:")
print(calculate_vif(X_fa_ohne_gdp.drop(columns='const')))



📉 VIF für RPPOP-Modell ohne NDGAIN:
                     Variable        VIF
11                         rl  11.766246
12                         cc   8.878340
4                      GRadUn   8.834811
10                         ge   8.175641
7                      GDPCap   6.478335
3                      CO2BIP   5.419000
6                    Ambition   3.896091
8                          va   3.251949
0   AdaptationMitigationRatio   3.123875
2                        LDCs   3.025041
9                          pv   2.559207
5                  AvgChange%   2.081650
1                        SIDS   1.623315
13                   δCO2/GDP   1.374667
14            Ambitionsfaktor   1.218074

📉 VIF für RPPOP-Modell ohne GDPCap:
                     Variable        VIF
7                      NDGAIN  16.583383
11                         rl  11.833566
12                         cc   8.439923
10                         ge   8.032134
4                      GRadUn   7.660683
3                      C

In [18]:

# CSV laden
df = pd.read_csv("NEWmergedpop.csv")

# Dummy-Variablen korrekt typisieren
df["SIDS"] = df["SIDS"].astype(int)
df["LDCs"] = df["LDCs"].astype(int)

# Funktion zur VIF-Berechnung
def calculate_vif(df_features):
    df_features = df_features.select_dtypes(include='number')
    vif_data = pd.DataFrame()
    vif_data["Variable"] = df_features.columns
    vif_data["VIF"] = [variance_inflation_factor(df_features.values, i)
                       for i in range(df_features.shape[1])]
    return vif_data.sort_values(by="VIF", ascending=False)

# Relevante Variablen definieren
all_vars = ['SIDS', 'LDCs', 'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
        #versuch ohne log
            'GDPCap', 'NDGAIN',
            'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']
vars_rp = ['AdaptationMitigationRatio'] + all_vars

# RPPOP ohne NDGAIN
X_rp_ohne_ndgain = df[vars_rp].drop(columns=["NDGAIN"]).dropna()
X_rp_ohne_ndgain = sm.add_constant(X_rp_ohne_ndgain)
print("\n📉 VIF für RPPOP-Modell ohne NDGAIN:")
print(calculate_vif(X_rp_ohne_ndgain.drop(columns='const')))

# RPPOP ohne LogGDPCap
X_rp_ohne_gdp = df[vars_rp].drop(columns=["GDPCap"]).dropna()
X_rp_ohne_gdp = sm.add_constant(X_rp_ohne_gdp)
print("\n📉 VIF für RPPOP-Modell ohne GDPCap:")
print(calculate_vif(X_rp_ohne_gdp.drop(columns='const')))

# FAPOP ohne NDGAIN
X_fa_ohne_ndgain = df[all_vars].drop(columns=["NDGAIN"]).dropna()
X_fa_ohne_ndgain = sm.add_constant(X_fa_ohne_ndgain)
print("\n📉 VIF für FAPOP-Modell ohne NDGAIN:")
print(calculate_vif(X_fa_ohne_ndgain.drop(columns='const')))

# FAPOP ohne LogGDPCap
X_fa_ohne_gdp = df[all_vars].drop(columns=["GDPCap"]).dropna()
X_fa_ohne_gdp = sm.add_constant(X_fa_ohne_gdp)
print("\n📉 VIF für FAPOP-Modell ohne GDPCap:")
print(calculate_vif(X_fa_ohne_gdp.drop(columns='const')))




📉 VIF für RPPOP-Modell ohne NDGAIN:
                     Variable       VIF
4                      GRadUn  8.277668
7                      GDPCap  7.030053
3                      CO2BIP  5.291275
6                    Ambition  3.446388
0   AdaptationMitigationRatio  2.828012
2                        LDCs  2.416164
5                  AvgChange%  2.018455
8                    Gov_PCA1  1.892720
1                        SIDS  1.510168
10                   δCO2/GDP  1.346638
11            Ambitionsfaktor  1.164336
9                    Gov_PCA2  1.150497

📉 VIF für RPPOP-Modell ohne GDPCap:
                     Variable        VIF
7                      NDGAIN  18.541749
4                      GRadUn   7.257712
3                      CO2BIP   5.775760
6                    Ambition   5.219776
0   AdaptationMitigationRatio   3.542846
2                        LDCs   2.482293
5                  AvgChange%   2.021333
8                    Gov_PCA1   1.796875
1                        SIDS   1.417

In [6]:
#Testweise Regression ohne logarithmus

# CSV-Datei laden
df = pd.read_csv("NEWmergedpop.csv")

# Relevante Spalten definieren
all_vars = [
    'RPPOP', 'FAPOP',
    'SIDS', 'LDCs', 
    'AdaptationMitigationRatio',  # Nur für RPPOP!
    'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
    'GDPCap', 'Gov_PCA1', 'Gov_PCA2','δCO2/GDP', 'Ambitionsfaktor'
]

# Datensatz vorbereiten
df_reg = df[all_vars].copy()
df_reg[['RPPOP', 'FAPOP']] = df_reg[['RPPOP', 'FAPOP']].fillna(0)
df_reg.dropna(inplace=True)
df_reg['SIDS'] = df_reg['SIDS'].astype(int)
df_reg['LDCs'] = df_reg['LDCs'].astype(int)

# Modell 1: RPPOP (mit AdaptationMitigationRatio)
X_rp = df_reg[['SIDS', 'LDCs', 'AdaptationMitigationRatio', 
               'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
               'GDPCap', 'Gov_PCA1', 'Gov_PCA2','δCO2/GDP', 'Ambitionsfaktor']]
X_rp = sm.add_constant(X_rp)
y_rp = df_reg['RPPOP']
model_rp = sm.OLS(y_rp, X_rp).fit()

# Modell 2: FAPOP (ohne AdaptationMitigationRatio)
X_fa = df_reg[['SIDS', 'LDCs', 
               'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
               'GDPCap','Gov_PCA1','Gov_PCA2','δCO2/GDP', 'Ambitionsfaktor']]
X_fa = sm.add_constant(X_fa)
y_fa = df_reg['FAPOP']
model_fa = sm.OLS(y_fa, X_fa).fit()

# Ergebnisse ausgeben
print("Regressionsergebnisse für RP Financing pro Einwohner:")
print(model_rp.summary())

print("\nRegressionsergebnisse für FA Financing pro Einwohner (ohne AdaptationMitigationRatio):")
print(model_fa.summary())


Regressionsergebnisse für RP Financing pro Einwohner:
                            OLS Regression Results                            
Dep. Variable:                  RPPOP   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     2.080
Date:                Fri, 20 Jun 2025   Prob (F-statistic):             0.0294
Time:                        08:23:47   Log-Likelihood:                -237.36
No. Observations:                  83   AIC:                             500.7
Df Residuals:                      70   BIC:                             532.2
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

In [24]:
#Test wie stark NDGAIN korreliert

# CSV-Datei laden
df = pd.read_csv("NEWmergedpop.csv")

# Liste relevanter Variablen
relevant_vars = [
    'NDGAIN', 'GDPCap', 'SIDS', 'LDCs', 'CO2BIP', 'GRadUn', 'AvgChange%', 
    'Ambition', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor'
]

# Subset des DataFrames erstellen
df_subset = df[relevant_vars]

# Korrelation berechnen
correlations = df_subset.corr(numeric_only=True)

# Ausgabe: sortierte Korrelation mit NDGAIN
print("🔍 Korrelation mit NDGAIN:")
print(correlations["NDGAIN"].sort_values(ascending=False))


🔍 Korrelation mit NDGAIN:
NDGAIN             1.000000
GDPCap             0.764231
Gov_PCA1           0.703445
GRadUn             0.479958
CO2BIP             0.227625
SIDS               0.151298
Ambition           0.045431
Ambitionsfaktor    0.039611
Gov_PCA2          -0.185415
δCO2/GDP          -0.312058
AvgChange%        -0.407103
LDCs              -0.692454
Name: NDGAIN, dtype: float64


📊 Breusch-Pagan-Test für log(RPPOP):
{'LM-Statistik': 19.690430501511745, 'p-Wert': 0.07317329429949485, 'F-Statistik': 1.8142730317396107, 'F p-Wert': 0.062347934308849204}


In [16]:
# NUR FÜR  BREUSCH BRAGAN TEST danach HC3

# CSV-Datei laden
df = pd.read_csv("NEWmergedpop.csv")

# Relevante Variablen definieren
all_vars = [
    'RPPOP', 'FAPOP',
    'SIDS', 'LDCs', 
    'AdaptationMitigationRatio',  # Nur für RPPOP
    'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
    'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor'
]

# Datensatz vorbereiten
df_reg = df[all_vars].copy()
df_reg[['RPPOP', 'FAPOP']] = df_reg[['RPPOP', 'FAPOP']].fillna(0)
df_reg.dropna(inplace=True)
df_reg['SIDS'] = df_reg['SIDS'].astype(int)
df_reg['LDCs'] = df_reg['LDCs'].astype(int)

# Zielvariablen log-transformieren
df_reg['log_RPPOP'] = np.log1p(df_reg['RPPOP'])
df_reg['log_FAPOP'] = np.log1p(df_reg['FAPOP'])

# Modell 1: log(RPPOP)
X_rp = df_reg[['SIDS', 'LDCs', 'AdaptationMitigationRatio', 
               'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
               'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']]
X_rp = sm.add_constant(X_rp)
y_rp = df_reg['log_RPPOP']
model_rp = sm.OLS(y_rp, X_rp).fit()

# Breusch-Pagan-Test für log(RPPOP)
residuals_rp = model_rp.resid
bp_test_rp = het_breuschpagan(residuals_rp, X_rp)
labels = ['LM-Statistik', 'p-Wert', 'F-Statistik', 'F p-Wert']
print("\n📊 Breusch-Pagan-Test für log(RPPOP):")
print(dict(zip(labels, bp_test_rp)))

# Modell 2: log(FAPOP)
X_fa = df_reg[['SIDS', 'LDCs', 
               'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
               'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']]
X_fa = sm.add_constant(X_fa)
y_fa = df_reg['log_FAPOP']
model_fa = sm.OLS(y_fa, X_fa).fit()

# Breusch-Pagan-Test für log(FAPOP)
residuals_fa = model_fa.resid
bp_test_fa = het_breuschpagan(residuals_fa, X_fa)
print("\n📊 Breusch-Pagan-Test für log(FAPOP):")
print(dict(zip(labels, bp_test_fa)))

# Ergebnisse ausgeben
print("\n📊 Regressionsergebnisse für log(RPPOP):")
print(model_rp.summary())

print("\n📊 Regressionsergebnisse für log(FAPOP):")
print(model_fa.summary())



📊 Breusch-Pagan-Test für log(RPPOP):
{'LM-Statistik': 19.690430501511745, 'p-Wert': 0.07317329429949485, 'F-Statistik': 1.8142730317396107, 'F p-Wert': 0.062347934308849204}

📊 Breusch-Pagan-Test für log(FAPOP):
{'LM-Statistik': 15.353315595083714, 'p-Wert': 0.16688654070959855, 'F-Statistik': 1.4649450192306444, 'F p-Wert': 0.16431562725640886}

📊 Regressionsergebnisse für log(RPPOP):
                            OLS Regression Results                            
Dep. Variable:              log_RPPOP   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     6.785
Date:                Fri, 20 Jun 2025   Prob (F-statistic):           6.25e-08
Time:                        10:27:37   Log-Likelihood:                -33.833
No. Observations:                  83   AIC:                             93.67
Df Residuals:                      70   BIC:             

In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# CSV-Datei laden
df = pd.read_csv("NEWmergedpop.csv")

# Relevante Variablen definieren
all_vars = [
    'RPPOP', 'FAPOP',
    'SIDS', 'LDCs', 
    'AdaptationMitigationRatio',  # Nur für RPPOP
    'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
    'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor'
]

# Datensatz vorbereiten
df_reg = df[all_vars].copy()
df_reg[['RPPOP', 'FAPOP']] = df_reg[['RPPOP', 'FAPOP']].fillna(0)
df_reg.dropna(inplace=True)
df_reg['SIDS'] = df_reg['SIDS'].astype(int)
df_reg['LDCs'] = df_reg['LDCs'].astype(int)

# Zielvariablen log-transformieren
df_reg['log_RPPOP'] = np.log1p(df_reg['RPPOP'])
df_reg['log_FAPOP'] = np.log1p(df_reg['FAPOP'])

# Modell 1: log(RPPOP)
X_rp = df_reg[['SIDS', 'LDCs', 'AdaptationMitigationRatio', 
               'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
               'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']]
X_rp = sm.add_constant(X_rp)
y_rp = df_reg['log_RPPOP']
model_rp = sm.OLS(y_rp, X_rp).fit(cov_type='HC3')

# Modell 2: log(FAPOP)
X_fa = df_reg[['SIDS', 'LDCs', 
               'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition', 
               'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']]
X_fa = sm.add_constant(X_fa)
y_fa = df_reg['log_FAPOP']
model_fa = sm.OLS(y_fa, X_fa).fit(cov_type='HC3')

# Ergebnisse ausgeben
print("\n📊 Regressionsergebnisse für log(RPPOP) mit robusten Fehlern (HC3):")
print(model_rp.summary())

print("\n📊 Regressionsergebnisse für log(FAPOP) mit robusten Fehlern (HC3):")
print(model_fa.summary())



📊 Regressionsergebnisse für log(RPPOP) mit robusten Fehlern (HC3):
                            OLS Regression Results                            
Dep. Variable:              log_RPPOP   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     4.357
Date:                Fri, 20 Jun 2025   Prob (F-statistic):           3.79e-05
Time:                        10:59:17   Log-Likelihood:                -33.833
No. Observations:                  83   AIC:                             93.67
Df Residuals:                      70   BIC:                             125.1
Df Model:                          12                                         
Covariance Type:                  HC3                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------

In [1]:
#Gemeinsame Regression von RPPOP und FAPOP als LOG
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad

# Daten laden und vorbereiten
df = pd.read_csv("NEWmergedpop.csv")
df['SumPOP'] = df['RPPOP'] + df['FAPOP']
df['log_SumPOP'] = np.log1p(df['SumPOP'])

# Nur relevante Variablen + Dummy-Kodierung
vars_x = ['SIDS', 'LDCs', 'CO2BIP', 'GRadUn', 'AvgChange%', 'Ambition',
          'GDPCap', 'Gov_PCA1', 'Gov_PCA2', 'δCO2/GDP', 'Ambitionsfaktor']
df['SIDS'] = df['SIDS'].astype(int)
df['LDCs'] = df['LDCs'].astype(int)
df_reg = df[['log_SumPOP'] + vars_x].dropna()

# VIF-Check
X = sm.add_constant(df_reg[vars_x])
vif_df = pd.DataFrame({
    'Variable': X.columns,
    'VIF': [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
print("VIF-Werte:")
print(vif_df)

# Regression
y = df_reg['log_SumPOP']
model = sm.OLS(y, X).fit(cov_type='HC3')
print("\nRegressionsergebnis für log(SumPOP) (HC3):")
print(model.summary())

# Normalverteilung der Residuen (Jarque-Bera)
jb = sm.stats.jarque_bera(model.resid)
print("\nJarque-Bera-Test auf Normalverteilung der Residuen:")
print(f"JB-Statistik: {jb[0]:.3f}, p-Wert: {jb[1]:.3g}, Skew: {jb[2]:.3f}, Kurtosis: {jb[3]:.3f}")

# Breusch-Pagan-Test auf Homoskedastizität
bp = het_breuschpagan(model.resid, X)
labels = ['LM-Statistik', 'p-Wert', 'F-Statistik', 'F p-Wert']
print("\nBreusch-Pagan-Test auf Homoskedastizität:")
print(dict(zip(labels, bp)))


VIF-Werte:
           Variable        VIF
0             const  15.428478
1              SIDS   1.265431
2              LDCs   1.889944
3            CO2BIP   1.672964
4            GRadUn   2.537957
5        AvgChange%   1.484894
6          Ambition   1.154211
7            GDPCap   3.183488
8          Gov_PCA1   1.799980
9          Gov_PCA2   1.266099
10         δCO2/GDP   1.408290
11  Ambitionsfaktor   1.110417

Regressionsergebnis für log(SumPOP) (HC3):
                            OLS Regression Results                            
Dep. Variable:             log_SumPOP   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.397
Method:                 Least Squares   F-statistic:                     3.819
Date:                Wed, 25 Jun 2025   Prob (F-statistic):           0.000190
Time:                        08:24:36   Log-Likelihood:                -118.32
No. Observations:                  94   AIC:                        